In [51]:
import struct, os
from Crypto.Cipher import AES
DNS_URL_POSTFIX = "dnsr.uk.to"
CODING_TABLE    = "IQJ6WORKEGL5B4YS3NZMPDHVCUA7FX2T"
ct = {}
for i,c in enumerate(CODING_TABLE.upper()):
    ct[c] = i
SECRET_KEY = bytes([0]*32)   # AES 256 secret key. should be actually 32 random bytes, not all zeros :p
#SECRET_KEY = os.urandom(32)  #That's better.
qn = "44Q23IJL2V24UDQBZTHRJD5SHJ.IPOF4V4EJMYQA4PMX6LAN622LW.M5OEVZSPUELORYQ7INILBS3B2Q.MQBBABAHHLVCJFNVSSO37LDD7Q.dnsr.uk.to."

### Add this to dnsSneaker.h

In [60]:
print("#define SECRET_KEY_256							{", end="")
for c in SECRET_KEY:
    print( "0x{0:02x},".format(c), end="")
print("}")

#define SECRET_KEY_256							{0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,}


### This is where the magic happens

In [53]:
def decodeBlock( messageBlock ):
    """ returns a bytes object """
    global ct
    res = 0
    for i,c in enumerate(messageBlock):
        res |= ct[c]<<(i*5)
    return res.to_bytes( 16, 'little' )

In [54]:
def dnsDecode( qnString ):
    global DNS_URL_POSTFIX, SECRET_KEY
    cipher = AES.new( SECRET_KEY, AES.MODE_ECB )
    if not qnString.endswith( '.' + DNS_URL_POSTFIX + '.'):
        raise RuntimeError("Bad hostname" + qnString.split(".")[-1])
    messageBlocks = qn.replace( '.' + DNS_URL_POSTFIX + '.', "" )
    resultBytes = bytearray()
    for messageBlock in messageBlocks.split("."):
        messageBlock.upper()
        resultBytes += cipher.decrypt( decodeBlock(messageBlock) )
    return resultBytes

In [58]:
res = dnsDecode( qn )
print(res.decode())

Wuhuu!, Communication seems fine!! :D :D So cool! 
	...
        
